In [1]:
import sys
sys.path.append('../')
from stim_surface_code.memory import MemoryPatch
from stim_surface_code import noise

# Basic usage

In [2]:
m = MemoryPatch(5, 5, 5)
m

--------------------------------------------------------
|    |    |X 25|    |    |    |X 26|    |    |    |    |
----------+----+--------------+----+--------------------
|    |   0|    |   1|    |   2|    |   3|    |   4|    |
----------+----+----+----+----+----+----+----+----+-----
|    |    |Z 27|    |X 28|    |Z 29|    |X 30|    |Z 31|
----------+----+----+----+----+----+----+----+----+-----
|    |   5|    |   6|    |   7|    |   8|    |   9|    |
-----+----+----+----+----+----+----+----+----+----------
|Z 32|    |X 33|    |Z 34|    |X 35|    |Z 36|    |    |
-----+----+----+----+----+----+----+----+----+----------
|    |L 10|    |L 11|    |L 12|    |L 13|    |L 14|    |
----------+----+----+----+----+----+----+----+----+-----
|    |    |Z 37|    |X 38|    |Z 39|    |X 40|    |Z 41|
----------+----+----+----+----+----+----+----+----+-----
|    |  15|    |  16|    |  17|    |  18|    |  19|    |
-----+----+----+----+----+----+----+----+----+----------
|Z 42|    |X 43|    |Z 44|    |

In [3]:
# 10x better noise vals than Google's device
noise_params = noise.GoogleNoiseParams.improve_noise_params(10)

print(noise_params.baseline_error_means)

{'T1': 0.0002, 'T2': 0.00030000000000000003, 'gate1_err': -5.0, 'gate2_err': -3.5, 'readout_err': -2.7}


In [4]:
m.set_error_vals_normal(noise_params.baseline_error_means, noise_params.baseline_error_stdevs)

In [5]:
m.count_logical_errors()

(0.0006491, 10000000)

# Increasing d

In [6]:
ds = [3,5,7,9]
for d in ds:
    m = MemoryPatch(d, d, d)
    m.set_error_vals_normal(noise_params.baseline_error_means, noise_params.baseline_error_stdevs)
    ler = m.count_logical_errors()[0]
    print(ler)

0.0089926
0.0006205
8.4e-05
1.1e-05
